# Housing prices in Hyderabad, India

## Project Objective 🎯

The objective of this project is to develop a regression model to predict housing prices in Hyderabad, India. Using features such as the property's area, location, number of bedrooms, and available amenities, the model will aim to estimate the market value of a property as accurately as possible.

- This predictive model will be a valuable tool for:
- Home Buyers and Sellers: To obtain an objective price estimate for a property.
- Real Estate Agents: To assist with property valuation and client advisory.
- Investors: To identify potentially undervalued or overvalued properties in the market.

## 4. Training the models

### 4.1 Loading the dataset

In [7]:
import pandas as pd
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from xgboost import XGBRegressor

from IPython.display import display


training_features = pd.read_parquet('../../datasets/processed/housing_prices/hyderabad_house_price_training_features.parquet')
training_labels = pd.read_parquet('../../datasets/processed/housing_prices/hyderabad_house_price_training_labels.parquet')

test_features = pd.read_parquet('../../datasets/processed/housing_prices/hyderabad_house_price_test_features.parquet')
test_labels= pd.read_parquet('../../datasets/processed/housing_prices/hyderabad_house_price_test_labels.parquet')

target_metrics = '../../datasets/processed/housing_prices/hyderabad_house_price_xgboost_metrics.csv'

### 4.2 Model Validation

**Problem:**

How to systematically find the best regression model and its optimal hyperparameters for our dataset.

**Justification:**

Since no single model is universally best (the "No Free Lunch" theorem), a systematic comparison is essential. GridSearchCV is the standard tool for this, as it exhaustively explores hyperparameter combinations and uses cross-validation to provide a robust estimate of model performance on unseen data.

**Action:**

The code iterates through a predefined list of models (e.g., Linear Regression, Lasso, Ridge) and their respective hyperparameter grids. For each model, it performs an exhaustive GridSearchCV with 10-fold cross-validation to find the best parameter set based on R² and RMSE metrics. All results are then compiled into a single Pandas DataFrame and saved to a CSV file for analysis.

**Verify:**

Success is verified by inspecting the final final_metrics DataFrame. It should contain the consolidated performance metrics for all evaluated models. The creation of the target CSV file also confirms that the process completed successfully.

In [8]:
folds = 10
max_components = training_features.shape[1]

scoring_metrics = {
    'neg_rmse': 'neg_root_mean_squared_error',
    'r2': 'r2'
}

models_configuration = [
    {
        'name': 'XGB', # XGBoost Regressor
        'pipeline': Pipeline([
            # Usamos XGBRegressor con parámetros base
            ('regresion', XGBRegressor(random_state=42, objective='reg:squarederror', n_jobs=-1)) 
        ]),
        'param_grid': {
            # Un grid de hiperparámetros de ejemplo para XGBoost
            'regresion__n_estimators': [100, 200, 300], # Número de árboles
            'regresion__learning_rate': [0.01, 0.1],     # Tasa de aprendizaje
            'regresion__max_depth': [3, 5, 7]             # Profundidad máxima de los árboles
        }
    }
]

all_metrics = []

for config in models_configuration:
    model_name = config['name']
    pipeline = config['pipeline']
    param_grid = config['param_grid']
    
    print(f"--- Executing: {model_name} ---")

    grid_search = GridSearchCV(
        estimator=pipeline,
        param_grid=param_grid,
        scoring=scoring_metrics,
        refit='r2',
        cv=folds,
        n_jobs=-1
    )

    grid_search.fit(training_features, training_labels)

    grid_metrics = pd.DataFrame(grid_search.cv_results_)
    grid_metrics['model_name'] = model_name
    
    all_metrics.append(grid_metrics)

final_metrics = pd.concat(all_metrics, ignore_index=True)
final_metrics['params'] = final_metrics['params'].astype(str)

final_metrics.to_csv(target_metrics)

display(final_metrics)

--- Executing: XGB ---


,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_regresion__learning_rate,param_regresion__max_depth,param_regresion__n_estimators,params,split0_test_neg_rmse,split1_test_neg_rmse,...,split4_test_r2,split5_test_r2,split6_test_r2,split7_test_r2,split8_test_r2,split9_test_r2,mean_test_r2,std_test_r2,rank_test_r2,model_name
0,3.285630,0.680436,0.017265,0.010688,0.01,3,100,"{'regresion__learning_rate': 0.01, 'regresion_...",-0.384908,-0.366500,...,0.739476,0.716406,0.585321,0.694804,0.730456,0.684932,0.691507,0.040038,18,XGB
1,3.631180,0.916536,0.109121,0.192378,0.01,3,200,"{'regresion__learning_rate': 0.01, 'regresion_...",-0.317647,-0.293684,...,0.844402,0.830526,0.684452,0.806551,0.833102,0.804503,0.800660,0.042203,15,XGB
2,0.195828,0.014878,0.011422,0.001141,0.01,3,300,"{'regresion__learning_rate': 0.01, 'regresion_...",-0.300088,-0.273168,...,0.865944,0.855001,0.708518,0.828641,0.853477,0.834006,0.825209,0.041933,12,XGB
3,0.119571,0.004623,0.011852,0.001062,0.01,5,100,"{'regresion__learning_rate': 0.01, 'regresion_...",-0.372698,-0.348987,...,0.764127,0.739364,0.613182,0.719801,0.753325,0.689700,0.710437,0.040105,17,XGB
4,0.216201,0.007331,0.013410,0.001798,0.01,5,200,"{'regresion__learning_rate': 0.01, 'regresion_...",-0.305867,-0.270832,...,0.868946,0.844747,0.709190,0.827863,0.853415,0.809447,0.817115,0.041380,14,XGB
5,0.287842,0.005817,0.012805,0.000974,0.01,5,300,"{'regresion__learning_rate': 0.01, 'regresion_...",-0.292593,-0.251343,...,0.887810,0.866829,0.730098,0.846115,0.870784,0.835311,0.838416,0.041159,11,XGB
6,0.292275,0.254885,0.013426,0.002835,0.01,7,100,"{'regresion__learning_rate': 0.01, 'regresion_...",-0.363528,-0.343532,...,0.776194,0.750956,0.628611,0.731301,0.757504,0.700992,0.719636,0.039342,16,XGB
7,0.427264,0.200749,0.013816,0.005255,0.01,7,200,"{'regresion__learning_rate': 0.01, 'regresion_...",-0.298540,-0.266008,...,0.877361,0.856305,0.737068,0.838975,0.858338,0.814807,0.825157,0.036951,13,XGB
8,0.518009,0.152548,0.012603,0.001171,0.01,7,300,"{'regresion__learning_rate': 0.01, 'regresion_...",-0.288221,-0.249144,...,0.895931,0.872959,0.755717,0.853219,0.872555,0.837873,0.843708,0.036470,10,XGB
9,0.096161,0.046746,0.011437,0.002371,0.10,3,100,"{'regresion__learning_rate': 0.1, 'regresion__...",-0.281018,-0.245883,...,0.899779,0.889059,0.740192,0.852538,0.880167,0.861090,0.854886,0.042471,8,XGB
